In [1]:
import dask.dataframe as ddf
import pandas as pd
import numpy as np
import re
import os
import time
from glob import glob

In [2]:
sr = pd.read_parquet('../../../result/TCM/TCM-sr')

In [3]:
hl = pd.read_parquet('../../../result/TCM/TCM-hl-with-editor')

In [4]:
group_pairs = pd.read_csv('result/TCM-mutual-and-min-revert.csv', sep="\t",encoding='utf-16')

In [5]:
group_pairs = group_pairs.sort_values('min_revert',ascending=False)

In [6]:
kept_group_pairs = group_pairs[group_pairs['min_revert']>=10]

In [7]:
# kept_group_pairs = group_pairs[group_pairs['min_revert']>=14]

In [8]:
kept_group_pairs = kept_group_pairs.drop(columns=['mutual_revert','min_revert'])

In [9]:
df = pd.read_csv('result/TCM-group-pair-article-mutual-revert-info.csv', sep="\t",encoding='utf-16')

In [10]:
df = kept_group_pairs.merge(df,on=['group1','group2'])

In [11]:
user_group = pd.read_csv('result/cluster-result-original-distance-0.2.csv', sep="\t",encoding='utf-16').set_index('contributor.username')

In [12]:
sr = sr.reset_index()

In [13]:
sr = sr.merge(user_group,left_on='contributor.username',right_index=True)

In [14]:
group_sr = sr.groupby(['group','page.title']).agg({'all_added_tokens':'sum','survived_tokens':'sum'})

In [15]:
group_sr['sr'] = group_sr['survived_tokens'] / group_sr['all_added_tokens']

In [16]:
group_sr = group_sr[['sr']]

In [17]:
hl = hl.merge(user_group,left_on='contributor.username',right_index=True)

In [18]:
group_hl = hl.groupby(['group','page.title']).agg({'hl':'mean'})

In [19]:
df = df.merge(group_sr.reset_index(),left_on=['group1','page.title'],right_on=['group','page.title'],how='left').rename(columns={'sr':'sr_group1'})
df = df.merge(group_sr.reset_index(),left_on=['group2','page.title'],right_on=['group','page.title'],how='left').rename(columns={'sr':'sr_group2'})

In [20]:
df = df.drop(columns={'group_x','group_y'})

In [21]:
df = df.merge(group_hl.reset_index(),left_on=['group1','page.title'],right_on=['group','page.title'],how='left').rename(columns={'hl':'hl_group1'})
df = df.merge(group_hl.reset_index(),left_on=['group2','page.title'],right_on=['group','page.title'],how='left').rename(columns={'hl':'hl_group2'})

In [22]:
df = df.drop(columns={'group_x','group_y'})

In [23]:
df['sr_quotient'] = df[['sr_group1','sr_group2']][(~np.isnan(df['sr_group1'])) & (~np.isnan(df['sr_group2']))].\
min(axis=1) / df[['sr_group1','sr_group2']].max(axis=1) 

In [24]:
df['hl_quotient'] = df[['hl_group1','hl_group2']][(~np.isnan(df['hl_group1'])) & (~np.isnan(df['hl_group2']))].\
min(axis=1) / df[['hl_group1','hl_group2']].max(axis=1) 

In [27]:
df.to_csv('result/TCM-sr-hl-for-top-ranked-pairs-using-wikiwho-new.csv', index=False,sep="\t",encoding='utf-16')